# Introduction to Folium Maps

For more on this library see https://python-visualization.github.io/folium/latest/getting_started.html
     

In [1]:
%matplotlib inline 

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import folium
import warnings
from IPython.display import display
#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0) # larger figure size
warnings.filterwarnings('ignore')

## Folium with Leaflet.js

Folium is a Python module wrapper for [Leaflet.js](http://leafletjs.com/), which uses [Open Street Maps](https://www.openstreetmap.us/). These are two, popular open source mapping libraries. Unlike Google maps API, its 100% free!

You can use Folium to render maps in Python and put data on the maps. Here's how easy it is to bring up a map:


In [2]:
CENTER_US = (39.8333333,-98.585522)
london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)
display(map)

You can zoom right down to the street level and get a amazing detail. There are different maps you can use, as was covered in this week's reading. 

### Mapping the student movie goers.

Let's take the largest category of movie goers and map their whereabouts. We will first need to import a data set to give us a lat/lng for the `zip_code` we have in the dataframe. We could look this up with Google's geolookup API, but that's too slow as we will be making 100's of requests. It's better to have them stored already and merge them with `goers`!

Let's import the zipcode database into a Pandas DataFrame, then merge it with the `goers` DataFrame:

In [4]:
# DATASET 1
goers = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/ist256/13-visualization/moviegoers.csv')
goers['age_group'] = ''
goers['age_group'][goers['age'] <=18] = 'Youth'
goers['age_group'][(goers['age'] >=19) & (goers['age'] <=55)] = 'Adult'
goers['age_group'][goers['age'] >=56] = 'Senior'

goers.sample(5)

,user_id,age,gender,occupation,zip_code,age_group
845,846,27,M,lawyer,47130,Adult
439,440,30,M,other,48076,Adult
798,799,49,F,administrator,19716,Adult
893,894,47,M,educator,74075,Adult
567,568,39,M,educator,1915,Adult


In [5]:
zipcodes = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/zipcodes/free-zipcode-database-Primary.csv', dtype = {'Zipcode' :object})
data = goers.merge(zipcodes,  how ='inner', left_on='zip_code', right_on='Zipcode')
students = data[ data['occupation'] == 'student']
students.sample(3)

,user_id,age,gender,occupation,zip_code,age_group,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
743,820,22,M,student,37725,Adult,37725,STANDARD,DANDRIDGE,TN,PRIMARY,36.02,-83.41,NA-US-TN-DANDRIDGE,False,7079.0,12966.0,2.067975e+08
123,99,20,M,student,63129,Adult,63129,STANDARD,SAINT LOUIS,MO,PRIMARY,38.63,-90.24,NA-US-MO-SAINT LOUIS,False,27488.0,48033.0,1.362870e+09
46,37,23,M,student,55105,Adult,55105,STANDARD,SAINT PAUL,MN,PRIMARY,44.94,-93.10,NA-US-MN-SAINT PAUL,False,13220.0,21132.0,8.058799e+08


Let's explain the code, as a Pandas refresher course:

1. in the first line I added `dtype = {'Zipcode' :object}` to force the `Zipcode` column to be of type `object` without that, it imports as type `int` and cannot match with the `goers` DataFrame.
1. the next line merges the two dataframes together where the `zip_code` in `goers` (on_left) matches `Zipcode` in `zipcodes` (on_right)
1. the result `data` is a combined DataFrame, which we then filter to only `student` occupations, sorting that in the `students` DataFrame


### Slapping those students on a map!

We're ready to place the students on a map. It's easy:

1. For each row in the students dataframe:
1.   get the coordinates (lat /lng )
1.   make a `marker` with the coordinates
1.   add the marker to the map with `add_children()`

Here we go!


In [6]:
for row in students.to_records():
    pos = (row['Lat'],row['Long'])
    message = f"{row['age']} year old {row['gender']}  {row['occupation']} from {row['City']},{row['State']}"
    marker = folium.Marker(location=pos, 
                    popup=message
                          )
    map.add_child(marker)
display(map)

## Interactive Maps

Let's combine the map with an Interact drop down. That way we select an occupation first then display the pins for only that occupation.

Here is an explanation of the code:

    lines 4-5  : create a unique list of occupations from the data frame
    lines 9-10 : create the map (must be re-created with each new selection
    lines 11   : apply pandas filter to the data frame based selection
    lines 12-18: loop over each row in the data frame, create a marker and add it to the map
    lines 19   : display the map


In [33]:
## todo write code here!
from ipywidgets import interact_manual

occupations_list = list(data['occupation'].dropna().unique())
occupations_list.sort()

@interact_manual(occupation=occupations_list)
def main(occupation):
    CENTER_US = (39.8333333,-98.585522)
    map = folium.Map(location=CENTER_US, zoom_start=4)
    filter_df = data[ data['occupation'] == occupation ]
    for row in filter_df.to_records():
        pos = (row['Lat'],row['Long'])
        message = f"{row['age']} year old {row['gender']}  {row['occupation']} from {row['City']},{row['State']}"
        marker = folium.Marker(location=pos, 
                        popup=message
                              )
        map.add_child(marker)
    display(filter_df)
    display(map)
    

interactive(children=(Dropdown(description='occupation', options=('administrator', 'artist', 'doctor', 'educat…